# 전통적인 머신러닝 방법을 사용한 텍스트 분류 파이프라인


<img src='https://raw.githubusercontent.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs/refs/heads/main/cover.png'
     alt="NLP와 LLM 실전 가이드(한빛미디어)"
     style="border: 3px solid gray; box-shadow: 5px 5px 15px rgba(0, 0, 0, 0.3); border-radius: 10px; width: 300px;"   width='300'>


* 저자:  
    - [Lior Gazit](https://www.linkedin.com/in/liorgazit).  
    - [Meysam Ghaffari](https://www.linkedin.com/in/meysam-ghaffari-ph-d-a2553088/).
* 역자:
    - [박조은](https://github.com/corazzon)
* 이 노트북은 다음의 책에서 소개하는 내용입니다.
    - 역서 : NLP와 LLM 실전 가이드(한빛미디어)
    - 원서 : [Mastering NLP from Foundations to LLMs](https://www.amazon.com/dp/1804619183)

colab 실습 :
https://github.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/Mastering-NLP-from-Foundations-to-LLMs/blob/main/Chapter5_notebooks/Ch5_Text_Classification_Traditional_ML.ipynb)  


원서 Colab 실습:   
https://github.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs   
<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/blob/liors_branch/Chapter5_notebooks/Ch5_Text_Classification_Traditional_ML.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

>*```면책사항: 이 노트북에서 다루는 내용과 아이디어는 저자들 개인의 것이며, 저자들의 고용주의 견해나 지적 재산을 대변하지 않습니다.```*

**목표: 트윗이 "기업 | 제품 뉴스"를 논하고 있는지 식별하기 위한 트윗 처리.**
이 노트북은 이진 분류기의 완전한 엔드-투-엔드 머신러닝 시스템 설계를 보여줍니다.
트윗의 원문이 주어졌을 때, 분류기는 어떤 트윗이 기업이나 제품 뉴스를 논하고 있는지 식별합니다.

## 파이프라인 구성:
1. 코드 설정
2. 데이터 수집
3. 데이터 처리
   1. 전처리
   2. 예비 데이터 탐색
   3. 특성 엔지니어링
      1. 새로운 수치형 특성 탐색
4. 훈련/테스트 분할
5. 예비 통계 분석 및 실현 가능성 연구
6. 특성 선택
7. 머신러닝
   1. ML 모델 반복
   2. 선택된 모델 생성
   3. 훈련 결과 생성
   4. 테스트 결과 생성


* 비고:
이는 단일 노트북 파일에 완전히 포함되도록 설계된 완전한 머신러닝 파이프라인입니다. 이는 교육 도구로 사용됨을 의도하고 있습니다.
전문적인 개발 환경에서는 재현성과 효율성을 위해 설계가 재현 가능한 `.py` 파일들로 작성되어야 합니다.

## 데이터:
[Hugging Face의 데이터셋:
twitter-financial-news-topic](https://huggingface.co/datasets/zeroshot/twitter-financial-news-topic):

>>
**Twitter Financial News 데이터셋은 금융 관련 트윗의 주석이 달린 말뭉치를 포함하는 영어 데이터셋입니다. 이 데이터셋은 금융 관련 트윗의 주제를 분류하는 데 사용됩니다.**

이 데이터셋은 20개의 레이블로 주석이 달린 21,107개의 문서를 포함합니다(참고로, 우리는 이 데이터셋의 레이블을 재지정하고 있습니다):
```
topics = {
    "LABEL_0": "애널리스트 업데이트",
    "LABEL_1": "연준 | 중앙은행",
    "**LABEL_2": "기업 | 제품 뉴스**", (참고: 우리는 이 레이블에 집중할 것입니다)
    "LABEL_3": "국채 | 회사채",
    "LABEL_4": "배당금",
    "LABEL_5": "실적",
    "LABEL_6": "에너지 | 석유",
    "LABEL_7": "금융",
    "LABEL_8": "통화",
    "LABEL_9": "일반 뉴스 | 의견",
    "LABEL_10": "금 | 금속 | 원자재",
    "LABEL_11": "IPO",
    "LABEL_12": "법률 | 규제",
    "LABEL_13": "M&A | 투자",
    "LABEL_14": "거시경제",
    "LABEL_15": "시장",
    "LABEL_16": "정치",
    "LABEL_17": "인사이동",
    "LABEL_18": "주식 논평",
    "LABEL_19": "주가 움직임",
}
```

**데이터는 Twitter API를 사용하여 수집되었습니다. 현재 데이터셋은 다중 클래스 분류 작업을 지원합니다.**

**요구사항:**
* Colab에서 실행할 때는 다음 런타임 노트북 설정을 사용하세요: `Python3, CPU`

>*```면책사항: 이 노트북에서 다루는 내용과 아이디어는 저자들 개인의 것이며, 저자들의 고용주의 견해나 지적 재산을 대변하지 않습니다.```*

설치 :

In [ ]:
# 비고(REMARK):
# 아래 코드가 Python 패키지 불일치로 인해 오류가 발생할 경우, 이는 새로운 버전의 업데이트로 인해 발생했을 가능성이 있습니다.
# 이러한 경우, "default_installations" 값을 False로 설정하여 기본 이미지를 복원하십시오:
default_installations = True

if default_installations:
    !pip -q install datasets num2words autocorrect
else:
    import requests
    text_file_path = "requirements__Ch5_Text_Classification_Traditional_ML.txt"
    url = "https://raw.githubusercontent.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/main/Chapter5_notebooks/" + text_file_path
    res = requests.get(url)
    with open(text_file_path, "w") as f:
        f.write(res.text)

    !pip install -r requirements__Ch5_Text_Classification_Traditional_ML.txt

Imports:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib

import scipy
import re
from datasets import load_dataset

from num2words import num2words
import nltk; nltk.download('punkt'); nltk.download('stopwords'); nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller

# 머신러닝 관련
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
import sklearn.linear_model as lm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

### Code Settings


In [ ]:
# 코드 설정:
# db_name: 원본 데이터를 보유한 HuggingFace 데이터베이스 이름
# do_preprocessing: 논리값, 데이터 전처리를 수행할지 여부
# do_enhanced_preprocessing: 논리값, 연산량이 큰 전처리를 수행할지 여부
# do_feature_eng: 논리값, 특성 공학을 수행할지 여부
# maximize_a_priori: 논리값, 사전 확률(a priori) 또는 사후 확률(a posteriori) 통계에 기반하여 초기 특성 선택을 최대화할지 여부
# num_chosen_features_per_class: 정수값, 초기 특성 선택 시 클래스별로 선택할 특성의 수
# test_size: 0에서 1 사이의 비율, 테스트 데이터셋 크기를 나타냄
# feature_eng_details: "TfidfVectorizer"(TF-IDF 특성 공학) 또는 "CountVectorizer"(원-핫 인코딩 중 하나)를 선택
# seed: 정수값, 결과의 재현성을 보장하기 위해 사용하는 랜덤 시드
config_dict = {'db_name': "zeroshot/twitter-financial-news-topic",
               'do_preprocessing': True,
               'do_enhanced_preprocessing': False,
               'do_feature_eng': True,
               'maximize_a_priori': True,
               'num_chosen_features_per_class': 200,
               'test_size': 0.2,
               'feature_eng_details': "CountVectorizer-binary",
               'ngram_range_min': 1,
               'ngram_range_max': 2,
               'max_features': 1000,
               'seed': 0}

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

### 데이터 수집

In [ ]:
dataset_raw = load_dataset(config_dict["db_name"])

### 데이터 처리  
머신러닝 모델을 학습시키고 예측하기 위해 데이터를 설정하는 단계입니다.

하나의 완전한 데이터프레임을 생성합니다.  
HuggingFace는 원래 데이터셋을 훈련(train)과 검증(validation) 두 개의 하위 집합으로 나누어 제공합니다.  
우리는 이를 하나로 병합한 후, 나중에 원하는 비율로 다시 분할할 예정입니다.  

In [ ]:
first_df = pd.DataFrame(dataset_raw["train"])
second_df = pd.DataFrame(dataset_raw["validation"])
dataset_df = pd.concat([first_df, second_df]).reset_index(drop=True)
dataset_df = dataset_df.rename(columns={"label": "_label_"})

원본 데이터를 일부 살펴봅니다:  

In [ ]:
dataset_df.head(10).style.set_properties(**{'text-align': 'left'})

In [ ]:
print("원본 레이블 분포:\n")
dataset_df[["_label_"]].value_counts()

우리는 특정 주제인 **"회사 | 제품 뉴스"**(레이블 2)에 초점을 맞출 것입니다.  
따라서 레이블을 다음과 같이 재분류할 예정입니다:  
>>
- 레이블 0: **회사 | 제품 뉴스가 아닌 경우**  
- 레이블 1: **회사 | 제품 뉴스인 경우**  

이제 분류 문제는 이진 분류 문제로 전환됩니다.  

In [ ]:
dataset_df_binary = dataset_df.copy()
dataset_df_binary["_label_"] = dataset_df_binary["_label_"].map({2:1}).fillna(0).map(int)

In [ ]:
print("새로운 레이블의 분포:\n")
frequencies = dataset_df_binary[["_label_"]].value_counts()
frequencies

In [ ]:
most_frequent_class = frequencies.index[:][0][0]
print("가장 빈도가 높은 클래스는:", most_frequent_class)
print("해당 클래스의 기준 정확도는:", round((dataset_df_binary["_label_"] == most_frequent_class).mean(), 3))

### 전처리

전처리 유틸리티 함수 정의:

In [ ]:
def digits_to_words(match):
    """
    문자열로 된 숫자를 영어 단어로 변환합니다. 이 함수는 기수(cardinal)와 서수(ordinal)를 구분합니다.
    예: "2" → "two", "2nd" → "second"

    입력: str
    출력: str
    """
    suffixes = ['st', 'nd', 'rd', 'th']
    # 이전 작업에 의존하지 않도록 소문자로 변환:
    string = match[0].lower()
    if string[-2:] in suffixes:
        type = 'ordinal'
        string = string[:-2]
    else:
        type = 'cardinal'

    return num2words(string, to=type)


def spelling_correction(text):
    """
    잘못된 철자를 올바른 철자로 교정합니다.

    입력: str
    출력: str
    """
    corrector = Speller()
    spells = [corrector(word) for word in text.split()]
    return " ".join(spells)


def remove_stop_words(text):
    """
    불용어(stopwords)를 제거합니다.

    입력: str
    출력: str
    """
    stopwords_set = set(stopwords.words('english'))
    return " ".join([word for word in text.split() if word not in stopwords_set])


def stemming(text):
    """
    각 단어에 대해 어간 추출(stemming)을 수행합니다.

    입력: str
    출력: str
    """
    stemmer = PorterStemmer()
    return " ".join([stemmer.stem(word) for word in text.split()])


def lemmatizing(text):
    """
    각 단어에 대해 표제어 추출(lemmatization)을 수행합니다.

    입력: str
    출력: str
    """
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


def preprocessing(input_text):
    """
    텍스트 전처리를 위한 전체 파이프라인을 나타냅니다.

    입력: str
    출력: str
    """
    output = input_text
    # 소문자로 변환:
    output = output.lower()
    # URL 제거:
    output = re.sub(r'http\S+', "", output)
    # 숫자를 단어로 변환:
    # 다음 정규 표현식은 연속된 숫자와 임시적으로 서수 접미사를 포함한 일치를 검색합니다:
    output = re.sub(r'\d+(st)?(nd)?(rd)?(th)?', digits_to_words, output, flags=re.IGNORECASE)
    # 구두점 및 기타 특수 문자 제거:
    output = re.sub('[^ A-Za-z0-9]+', '', output)

    if config_dict["do_enhanced_preprocessing"]:
        # 철자 교정:
        output = spelling_correction(output)

    # 불용어 제거:
    output = remove_stop_words(output)

    if config_dict["do_enhanced_preprocessing"]:
        # 어간 추출:
        output = stemming(output)
        # 표제어 추출:
        output = lemmatizing(output)

    return output

전처리 수행:

In [ ]:
dataset_clean = dataset_df_binary.copy()
if config_dict["do_preprocessing"]:
    dataset_clean["text"] = [preprocessing(text) for text in dataset_clean["text"]]

In [ ]:
dataset_clean.head(10).style.set_properties(**{'text-align': 'left'})

## 초기 데이터 탐색  

모든 머신러닝 프로젝트는 데이터를 기본적으로 탐색하는 작업에서 시작해야 합니다.  
그 주요 목표는 데이터의 특성을 파악하고, "X" 데이터(이 경우, 트윗 텍스트)와 원하는 값 "Y"(이 경우, 주제 분류) 간의 연관성을 연구하는 것입니다.  

우선, 데이터의 가장 단순한 특성 중 하나인 각 트윗의 길이를 살펴보는 것부터 시작하겠습니다.  
이후에는 사용된 언어와 주제 레이블 간의 통계적 의존성을 탐구할 예정입니다.  

In [ ]:
!pip install -Uq koreanize-matplotlib
import koreanize_matplotlib

In [ ]:
dataset_clean["length of text"] = dataset_clean["text"].map(len)
ax = dataset_clean.plot.hist(column=["length of text"], by="_label_", bins=50, alpha=0.5, figsize=(10, 6),
                             title="클래스별 트윗 문자열 길이 분포", xlim=[0, 1000])

## 특성 공학

In [ ]:
def feat_eng_text_df(in_df, text_col, labels_col, config_dict):
    if "CountVectorizer-binary" == config_dict["feature_eng_details"]:
        # 특성 공학 방법: 이진 (원-핫 인코딩)
        print("특성 공학 방법: 이진 (원-핫 인코딩)")
        countvectorizer = CountVectorizer(ngram_range=(config_dict["ngram_range_min"], config_dict["ngram_range_max"]),
                                          stop_words='english',
                                          max_features=config_dict["max_features"],
                                          binary=True)

    elif "CountVectorizer-BOW" == config_dict["feature_eng_details"]:
        # 특성 공학 방법: 단어 가방(Bag of Words)
        print("특성 공학 방법: 단어 가방(Bag of Words)")
        countvectorizer = CountVectorizer(ngram_range=(config_dict["ngram_range_min"], config_dict["ngram_range_max"]),
                                          stop_words='english',
                                          max_features=config_dict["max_features"],
                                          binary=False)

    out_arr = countvectorizer.fit_transform(in_df[text_col])
    count_tokens = countvectorizer.get_feature_names_out()
    out_df = pd.DataFrame(data=out_arr.toarray(), columns=count_tokens)
    out_df[labels_col] = list(in_df[labels_col])
    return out_df


if config_dict["do_feature_eng"]:
    dataset_feat_eng = feat_eng_text_df(dataset_clean, 'text', '_label_', config_dict)
else:
    # 주의: 특성 공학을 건너뛰는 옵션입니다.
    # 이 옵션은 BERT, GPT 등 자체적으로 텍스트를 처리하는
    # 딥러닝 언어 모델을 사용할 때만 선택하세요.
    # 일반적인 머신러닝 모델 사용시 특성 공학은 필수입니다.
    dataset_feat_eng = dataset_clean.copy()

## 새로운 수치형 특성 탐색

In [ ]:
dataset_feat_eng.head()

In [ ]:
dataset_feat_eng.describe().loc[['min', 'max', 'mean']]

## Train/Test 데이터 세트 나누기

In [ ]:
dataset_feat_eng_test = dataset_feat_eng.sample(frac=config_dict["test_size"],random_state=config_dict['seed'])
dataset_feat_eng_train = dataset_feat_eng.drop(dataset_feat_eng_test.index)
dataset_feat_eng_test.shape, dataset_feat_eng_train.shape

## 초기 통계 분석 및 실현 가능성 연구

이 과정은 머신러닝을 적용하기 전에 수행하는 초기 연구에서 가장 가치 있는 단계 중 하나입니다.  
이 단계에서는 "X"와 "Y" 간의 관계를 측정하여 "상관관계"가 존재하는지 확인합니다.

만약 X와 Y가 모두 수치 데이터이고 회귀 문제라면, X와 Y 간의 상관관계를 평가하여 선형 회귀 모델이 좋은 결과를 낼 가능성을 확인하는 것이 합리적일 것입니다.

그러나 X와 Y 모두 본질적으로 수치 데이터가 아니므로, 두 변수 간의 **통계적 의존성**을 평가하여 모델이 탐지할 수 있는 "신호"가 존재하는지 판단하려고 합니다.  

계산:  
**P(feature | class)**

In [ ]:
# 클래스별 특성 통계:
means_by_class = dataset_feat_eng_train.groupby(by=["_label_"]).mean().T.sort_index()
means_by_class.head()

통계적 의존성을 반영하는 비율을 계산합니다:  
**P(class, feature)/(P(class)P(feature))**  
이것은 다음과 같이 다시 쓸 수 있습니다:  
**P(class | feature)/P(class)**  
또는 동등하게:  
**P(feature | class)/P(feature)**  

\*참고:  
아래 계산은 각 텍스트 항목의 수치적 특성이 **이진(binary)**인 경우에만 확률 측정으로 간주됩니다.  
만약 Bag of Words(BOW)나 TF/IDF와 같은 다른 특성 방법을 사용한다면, 아래 값은 확률이 아니라 그 대략적인 값을 나타냅니다.  

In [ ]:
P_class = sorted([[c, np.mean(dataset_feat_eng["_label_"] == c)] for c in set(means_by_class.columns)])
P_feature = sorted([[f, np.mean(dataset_feat_eng[f] > 0)] for f in dataset_feat_eng.columns if f != "_label_"])
P_feature_inv = [[f, 1/p] for f, p in P_feature]

P_class_arr = np.array(P_class)
P_feature_arr = np.array(P_feature)
P_feature_inv_arr = np.array(P_feature_inv)
# 특성 확률의 "열 벡터"와 클래스 확률의 "행 벡터"를 곱하여
# 각 요소가 확률의 곱으로 구성된 행렬을 생성합니다.
P_class_prod_P_feature_inv_arr = np.outer(P_feature_inv_arr[:, 1].astype(float), P_class_arr[:, 1].astype(float))

P_class_given_feature = means_by_class.copy()
for feature_counter in range(len(P_class_given_feature)):
  for c in P_class_given_feature.columns:
    # 오른쪽 항: P(feature | class) / P(feature)
    P_class_given_feature[c][feature_counter] = means_by_class[c][feature_counter] / P_feature_arr[feature_counter, 1].astype(float)

**클래스 "0"을 가장 잘 나타내는 단어들:**

In [ ]:
P_class_given_feature.sort_values([0], ascending=False)[[0]].head(10)

**클래스 "1"을 가장 잘 나타내는 단어들:**

In [ ]:
P_class_given_feature.sort_values([1], ascending=False)[[1]].head(10)

## 특성 선택

이 과정은 단변량 특성 선택(univariate feature selection)입니다.  
특성 값이 0/1일 때와 클래스 값이 0/1일 때의 조건부 의존성을 기반으로 하며, 특성의 평균 값이 해당 특성의 확률을 나타냅니다.  
특성 선택 과정은 **훈련 데이터셋에서만** 수행된다는 점에 유의하세요.  

각 클래스에 대해 가장 특징적인 특성을 선택합니다.  
다음 중 하나를 극대화합니다:  
* 사전 확률 분포 $ P(\text{feature} | \text{class})$, 최대 가능도(Max Likelihood)  
또는  
* 사후 확률 $ P(\text{class} | \text{feature})$, 최대 사후 확률(MAP)  

In [ ]:
chosen_features = []
if config_dict["maximize_a_priori"] == True:
    classes = means_by_class.columns
    for c in classes:
        chosen_features += list(means_by_class[c].sort_values(ascending=False).index[:config_dict["num_chosen_features_per_class"] + 1])
else:
    classes = P_class_given_feature.columns
    for c in classes:
        chosen_features += list(P_class_given_feature[c].sort_values(ascending=False).index[:config_dict["num_chosen_features_per_class"] + 1])

chosen_features = list(set(chosen_features))

In [ ]:
chosen_features[:20]

### 선택된 특성만 남기기:  
훈련 데이터를 기반으로 어떤 특성이 "중요한지"를 도출했으므로, 이 특성들을 훈련 데이터와 테스트 데이터 모두에 적용하여 선택합니다.  

In [ ]:
dataset_feat_eng_train_selected = dataset_feat_eng_train.filter(chosen_features + ["_label_"])
dataset_feat_eng_test_selected = dataset_feat_eng_test.filter(chosen_features + ["_label_"])

dataset_feat_eng_train_selected.head()

In [ ]:
dataset_feat_eng_train_selected["_label_"].value_counts()

# 머신러닝

In [ ]:
dataset_feat_eng_train_selected.head()

데이터셋에서 Y 레이블을 추출하고, 모델에 적합하도록 변수 유형을 변환합니다.

In [ ]:
x_features_train = dataset_feat_eng_train_selected.values[:, 0:-1]
y_labels_train = dataset_feat_eng_train_selected.values[:, -1]

x_features_test = dataset_feat_eng_test_selected.values[:, :-1]
y_labels_test = dataset_feat_eng_test_selected.values[:, -1]

x_features_train.shape, y_labels_train.shape, x_features_test.shape, y_labels_test.shape

#### 머신러닝 모델 반복 실험  

In [ ]:
models = []
models.append(("Random Forest", RandomForestClassifier(random_state=config_dict['seed'])))
models.append(("LASSO", lm.LogisticRegression(solver='liblinear', penalty='l1', max_iter=1000, random_state=config_dict['seed'])))
models.append(("KNN", KNeighborsClassifier()))
models.append(("Decision Tree", DecisionTreeClassifier(random_state=config_dict['seed'])))
models.append(("SVM", SVC(gamma='auto', random_state=config_dict['seed'])))

results = []
names = []
best_mean_result = 0
best_std_result = 0
for name, model in models:
    kfold = StratifiedKFold()
    cv_results = cross_val_score(model, x_features_train, y_labels_train, scoring='accuracy', cv=kfold)
    results.append(cv_results)
    names.append(name)
    print(name + ": mean(accuracy)=" + str(round(np.mean(cv_results), 3)) + ", std(accuracy)=" + str(round(np.std(cv_results), 3)))
    if (best_mean_result < np.mean(cv_results)) or \
        ((best_mean_result == np.mean(cv_results)) and (best_std_result > np.std(cv_results))):
        best_mean_result = np.mean(cv_results)
        best_std_result = np.std(cv_results)
        best_model_name = name
        best_model = model
print("\n최적의 모델은:\n" + best_model_name)

검증 폴드별 결과 분포를 관찰합니다:

In [ ]:
matplotlib.pyplot.boxplot(results, labels=names)
matplotlib.pyplot.title("Models' results' distribution of accuracy")
matplotlib.pyplot.show()

### 선택한 모델 생성  


하이퍼파라미터 최적화:

In [ ]:
model = lm.LogisticRegression(solver='liblinear', penalty='l1', max_iter=1000, random_state=config_dict['seed'])
params = {"C": np.linspace(start=0.001, stop=10, num=20)}
grid_search = GridSearchCV(model, params, scoring='accuracy')
grid_search.fit(x_features_train, y_labels_train)
print("최적의 하이퍼파라미터 'C' 값은:", grid_search.best_params_["C"])

최적화된 모델을 훈련 데이터셋에 적합합니다:

In [ ]:
model = lm.LogisticRegression(C=grid_search.best_params_["C"], max_iter=1000, random_state=config_dict['seed'])
model.fit(x_features_train, y_labels_train)

In [ ]:
# These two cells are for if we wanted to pick the Random Forest model:

# model = RandomForestClassifier()
# params = {"n_estimators": range(10, 31, 3),
#           "min_samples_split": range(2, 10, 2)}
# grid_search = GridSearchCV(model, params)
# grid_search.fit(x_features_train, y_labels_train)

# print("The optimal hyperparameter 'n_estimators' is:", grid_search.best_params_["n_estimators"])
# print("The optimal hyperparameter 'min_samples_split' is:", grid_search.best_params_["min_samples_split"])


In [ ]:
# model = RandomForestClassifier(n_estimators=grid_search.best_params_["n_estimators"],
#                                min_samples_split=grid_search.best_params_["min_samples_split"],
#                                )
# model.fit(x_features_train, y_labels_train)

### 훈련 결과 생성: 설계 선택에 활용

In [ ]:
y_train_estimated = model.predict(x_features_train)
accuracy_train = np.mean(y_train_estimated == y_labels_train)
baseline_accuracy_train = np.mean(0 == y_labels_train)
accuracy_lift_train = 100 * (accuracy_train/baseline_accuracy_train - 1)

print("훈련 세트 결과:\n-------------------------")
print("기준 모델(더미 분류기) 정확도:", round(baseline_accuracy_train, 2))
print("현재 모델의 정확도:", round(accuracy_train, 2))
print("정확도 향상은:", round(accuracy_lift_train), "%")

### 테스트 결과 생성: 성능 설명에 활용

In [ ]:
y_test_estimated = model.predict(x_features_test)
accuracy_test = np.mean(y_test_estimated == y_labels_test)
baseline_accuracy_test = np.mean(0 == y_labels_test)
accuracy_lift = 100 * (accuracy_test/baseline_accuracy_test - 1)

print("테스트 세트 결과:\n-------------------------")
print("기준 모델(더미 분류기) 정확도:", round(baseline_accuracy_test, 2))
print("현재 모델의 정확도:", round(accuracy_test, 2))
print("정확도 향상은:", round(accuracy_lift), "%")

print("\n혼동 행렬(Confusion Matrix):")
print(confusion_matrix(y_labels_test, y_test_estimated))
print("\n분류 보고서(Classification Report):")
print(classification_report(y_labels_test, y_test_estimated))